In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[9, 'Hercy', None, 43], [6, 'Alice', 9, 41], [4, 'Bob', 9, 36], [2, 'Winston', None, 37]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'reports_to', 'age']).astype({'employee_id':'Int64', 'name':'object', 'reports_to':'Int64', 'age':'Int64'})

In [4]:
employees = spark.createDataFrame(employees)
employees.show()

+-----------+-------+----------+---+
|employee_id|   name|reports_to|age|
+-----------+-------+----------+---+
|          9|  Hercy|      NULL| 43|
|          6|  Alice|         9| 41|
|          4|    Bob|         9| 36|
|          2|Winston|      NULL| 37|
+-----------+-------+----------+---+



In [10]:
from pyspark.sql.functions import count, avg, col, round

employees \
.groupby('reports_to') \
.agg(count('employee_id').alias('reports_count'),
     avg('age').alias('average_age')) \
.withColumn('average_age', round(col('average_age'))) \
.withColumnRenamed('reports_to', 'employee_id') \
.join(employees,
      'employee_id',
      'inner') \
.select(['employee_id', 'name', 'reports_count', 'average_age']) \
.show()


+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|       39.0|
+-----------+-----+-------------+-----------+

